In [1]:
import cv2
import numpy as np
import time
from functions import CalculateWeight, InvertImage, Mark4
from performance_measures import ThinningRate, ThinningSpeed


In [63]:
def kmm(image,colouredImage,imageName):
    

    n, m = len(image), len(image[0])
    image[image<=128] = 0
    image[image>128] = 255
    image[image==255] = 1

    # INITIALISATION
    change = True
    iterations = 0
    start = time.time()

    while change:
        change = False

        # MARKING 2's
        for i in range(n):
            for j in range(m):
                if image[i][j]!=0:
                    if i>0 and image[i-1][j]==0:
                        image[i][j] = 2
                    elif j>0 and image[i][j-1]==0:
                        image[i][j] = 2
                    elif i<n-1 and image[i+1][j]==0:
                        image[i][j] = 2
                    elif j<m-1 and image[i][j+1]==0:
                        image[i][j] = 2

        # MARKING 3's
        for i in range(n):
            for j in range(m):
                if image[i][j]==1:
                    if i>0 and j>0 and image[i-1][j-1]==0:
                        image[i][j] = 3
                    elif i<n-1 and j>0 and image[i+1][j-1]==0:
                        image[i][j] = 3
                    elif i<n-1 and j<m-1 and image[i+1][j+1]==0:
                        image[i][j] = 3
                    elif i>0 and j<m-1 and image[i-1][j+1]==0:
                        image[i][j] = 3

        # MARKING 4's
        for i in range(n):
            for j in range(m):
                if image[i][j]!=0:
                    count = Mark4(i, j, image)
                    if count==2 or count==3 or count==4:
                        image[i][j] = 4

        # DELETING UNNECESSARY 4's
        for i in range(n):
            for j in range(m):
                if image[i][j]==4:
                    if CalculateWeight(i, j, image) in deletion_array:
                        image[i][j] = 0
                    else:
                        image[i][j] = 1

        # DELETING UNNECESSARY 2's
        for i in range(n):
            for j in range(m):
                if image[i][j]==2:
                    if CalculateWeight(i, j, image) in deletion_array:
                        image[i][j] = 0
                        change = True
                    else:
                        image[i][j] = 1

        # DELETING UNNECESSARY 3's
        for i in range(n):
            for j in range(m):
                if image[i][j]==3:
                    if CalculateWeight(i, j, image) in deletion_array:
                        image[i][j] = 0
                        change = True
                    else:
                        image[i][j] = 1

        # image = image*255
        # cv2.imshow("image", image)
        # cv2.waitKey(100)
        # image = image//255

    exec_time = time.time()-start
    print ("Execution Time For the Image : ", exec_time)

   
#     og_image[og_image<=128] = 0
#     og_image[og_image>128] = 255
#     image = image*255
#     final_image = og_image-image
#     cv2.imshow("image", final_image)
#     cv2.imwrite("Output/"+name()[7:-4]+"_kmm.png", final_image)
#     cv2.waitKey(1000)

    image2 = image.copy()
    image2*=255
    mask_inv = cv2.bitwise_not(image2)

    og_image = colouredImage
    img1_bg = cv2.bitwise_and(og_image,og_image,mask = mask_inv)
    # og_image[og_image<=128] = 0
    # og_image[og_image>128] = 255

    image2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB)
    final_image = cv2.add(img1_bg,image2)
    # final_image = og_image-image
#     cv2.imshow("output",final_image)
#     cv2.imwrite(path+name[:-4]+"_kmm.png", final_image)
#     cv2.waitKey()
    cv2.imwrite("kmm"+imageName+".jpg",final_image)
    og_image[og_image<=128] = 0
    og_image[og_image>128] = 1
    og_image *= 255
    tr = ThinningRate(image)
    op, sp, ts = ThinningSpeed(og_image, image*255, exec_time)
    print ("Name : ", imageName)
    print ("Algo : KMM")
    print ("Thinning Rate : ", tr)
    print ("Object Points : ", op)
    print ("Skeleton Points : ", sp)
    print ("Thinning Speed : ", ts)

In [61]:
def k3m(image,colouredImage,imageName):
    
   
    change = True

    # BACKGROUND = 0, FOREGROUND = 1
    n, m = len(image), len(image[0])
    image[image<=128] = 0
    image[image>128] = 255
    image[image==255] = 1

    start = time.time()
    iteration = 1
    while change:
        change = False

        # IDENTIFYING BORDER PIXELS
        borders = set()
        for i in range(n):
            for j in range(m):
                if image[i][j] and CalculateWeight(i, j, image) in border_lookup:
                    borders.add((i, j))

        # REMOVING PIXELS IN PHASES
        ind = 0
        while ind<len(lookup_arrays):
            curr_lookup = lookup_arrays[ind]
            to_be_removed = set()
            for pixel in borders:
                if CalculateWeight(pixel[0], pixel[1], image) in curr_lookup:
                    image[pixel[0]][pixel[1]] = 0
                    change = True
                    to_be_removed.add(pixel)
            for pixel in to_be_removed:
                borders.remove(pixel)
            ind += 1

        # VISUALISATION
        # image = image*255
        # cv2.imshow("image", image)
        # cv2.waitKey(100)
        # image = image//255


    # ONE PIXEL THINNING
    for i in range(n):
        for j in range(m):
            if image[i][j] and CalculateWeight(i, j, image) in one_pixel_lookup:
                image[i][j] = 0

    exec_time = time.time()-start
    print ("Execution Time For the Image : ", exec_time)

#     og_image = cv2.imread(name(), 0)
#     og_image[og_image<=128] = 0
#     og_image[og_image>128] = 255
#     image = image*255
#     final_image = og_image-image
#     cv2.imshow("image", final_image)
#     cv2.imwrite("Output/"+name()[7:-4]+"_k3m.png", final_image)
#     cv2.waitKey(1000)

    image2 = image.copy()
    image2*=255
    mask_inv = cv2.bitwise_not(image2)

    og_image = colouredImage
    img1_bg = cv2.bitwise_and(og_image,og_image,mask = mask_inv)
    # og_image[og_image<=128] = 0
    # og_image[og_image>128] = 255

    image2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB)
    final_image = cv2.add(img1_bg,image2)
    # final_image = og_image-image
    
    print(image[image==1].shape)
    
    cv2.imshow("Output",final_image)
    cv2.imwrite("k3m"+imageName+".jpg",final_image)
#     cv2.imwrite(path+name[:-4]+"_kmm.png", final_image)
    cv2.waitKey()
    og_image[og_image<=128] = 0
    og_image[og_image>128] = 1
    og_image *= 255
    tr = ThinningRate(image)
    op, sp, ts = ThinningSpeed(og_image, image*255, exec_time)
    print ("Name : ", imageName)
    print ("Algo : K3M")
    print ("Thinning Rate : ", tr)
    print ("Object Points : ", op)
    print ("Skeleton Points : ", sp)
    print ("Thinning Speed : ", ts)

In [56]:
def zhangSuen(image,colouredImage,imageName):
    
    def thinningZS(im):
        img = np.copy(im)
        prev = np.zeros(img.shape,np.uint8)
        while True:
            img = thinningZSIteration(img,0)
            img = thinningZSIteration(img,1)
            diff = np.sum(np.abs(prev-img))
            if not diff:
                break
            prev = img
        return img

    def thinningZSIteration(img, iter):
        marker = np.zeros(img.shape,np.uint8);
        for i in range(1,img.shape[0]-1):
            for j in range(1,img.shape[1]-1):
                p2 = img[(i-1),j]
                p3 = img[(i-1),j+1]
                p4 = img[(i),j+1]
                p5 = img[(i+1),j+1]
                p6 = img[(i+1),j]
                p7 = img[(i+1),j-1]
                p8 = img[(i),j-1]
                p9 = img[(i-1),j-1]
                A  = (p2 == 0 and p3) + (p3 == 0 and p4) + (p4 == 0 and p5) + (p5 == 0 and p6) + (p6 == 0 and p7) + (p7 == 0 and p8) + (p8 == 0 and p9) + (p9 == 0 and p2)
                B  = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
                m1 = (p2 * p4 * p6) if (iter == 0 ) else (p2 * p4 * p8)
                m2 = (p4 * p6 * p8) if (iter == 0 ) else (p2 * p6 * p8)
                if (A == 1 and (B >= 2 and B <= 6) and m1 == 0 and m2 == 0):
                    marker[i,j] = 1;
        return np.bitwise_and(img,np.bitwise_not(marker))

    og_image = image
    og_image = (og_image[:,:]>128).astype(np.uint8)
    start = time.time()
    image = thinningZS(og_image)
    exec_time = time.time()-start
#     og_image = og_image*255
#     final_image = og_image-image
    print ("Execution Time For the Image : ", exec_time)

#     cv2.imshow("cool", final_image)
#     cv2.waitKey(2000)
#     cv2.imwrite("Output/"+name()[7:-4]+"_zs.png", final_image)

    image2 = image.copy()
    image2*=255
    mask_inv = cv2.bitwise_not(image2)

    og_image = colouredImage
    img1_bg = cv2.bitwise_and(og_image,og_image,mask = mask_inv)
    # og_image[og_image<=128] = 0
    # og_image[og_image>128] = 255

    image2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB)
    final_image = cv2.add(img1_bg,image2)
    # final_image = og_image-image
    
    print(image[image==1].shape)
    cv2.imwrite("zs"+imageName+".jpg",final_image)
#     cv2.imshow("Output",final_image)
#     cv2.imwrite(path+name[:-4]+"_kmm.png", final_image)
#     cv2.waitKey()
    og_image[og_image<=128] = 0
    og_image[og_image>128] = 1
    og_image *= 255
    tr = ThinningRate(image)
    op, sp, ts = ThinningSpeed(og_image, image*255, exec_time)
    print ("Name : ", imageName)
    print ("Algo : ZS")
    print ("Thinning Rate : ", tr)
    print ("Object Points : ", op)
    print ("Skeleton Points : ", sp)
    print ("Thinning Speed : ", ts)

In [57]:
def guoHall(image,colouredImage,imageName):
    def thinningGHIteration(img, iter):
        marker = np.zeros(img.shape,np.uint8);
        for i in range(1,img.shape[0]-1):
            for j in range(1,img.shape[1]-1):
                p2 = img[(i-1),j]
                p3 = img[(i-1),j+1]
                p4 = img[(i),j+1]
                p5 = img[(i+1),j+1]
                p6 = img[(i+1),j]
                p7 = img[(i+1),j-1]
                p8 = img[(i),j-1]
                p9 = img[(i-1),j-1]
                c = (p2==0 and (p3 or p4)) + (p4==0 and (p5 or p6)) + (p6==0 and (p7 or p8)) + (p8==0 and (p9 or p2))
                n1 = (p9 or p2) + (p3 or p4) + (p5 or p6) + (p7 or p8)
                n2 = (p2 or p3) + (p4 or p5) + (p6 or p7) + (p8 or p9)
                if(n1 < n2):
                    n = n1
                else:
                    n = n2
                if(iter == 0):
                    m = ((p6 or p7 or p9==0) and p8)
                else:
                    m = ((p2 or p3 or p5==0) and p4)
                if(c == 1 and (n>=2 and n<=3) and m==0):
                    marker[i][j] = 1
        return np.bitwise_and(img, np.bitwise_not(marker))

    def thinningGH(im):
        img = np.copy(im)
        prev = np.zeros(img.shape, np.uint8)
        while(True):
            img = thinningGHIteration(img, 0)
            img = thinningGHIteration(img, 1)
            diff = np.sum(np.abs(prev-img))
            if not diff:
                break
            prev = img 
        return img

    og_image = image
    og_image = (og_image[:,:]>128).astype(np.uint8)
    start = time.time()
    image = thinningGH(og_image)
    exec_time = time.time()-start
#     og_image = og_image*255
#     final_image = og_image-image
    print ("Execution Time For the Image : ", exec_time)

#     cv2.imshow("cool", final_image)
#     cv2.waitKey(2000)
#     cv2.imwrite("Output/"+name()[7:-4]+"_zs.png", final_image)

    image2 = image.copy()
    image2*=255
    mask_inv = cv2.bitwise_not(image2)

    og_image = colouredImage
    img1_bg = cv2.bitwise_and(og_image,og_image,mask = mask_inv)
    # og_image[og_image<=128] = 0
    # og_image[og_image>128] = 255

    image2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB)
    final_image = cv2.add(img1_bg,image2)
    # final_image = og_image-image
    
    print(image[image==1].shape)
    cv2.imwrite("gh"+imageName+".jpg",final_image)
#     cv2.imshow("Output",final_image)
#     cv2.imwrite(path+name[:-4]+"_kmm.png", final_image)
#     cv2.waitKey()
    og_image[og_image<=128] = 0
    og_image[og_image>128] = 1
    og_image *= 255
    tr = ThinningRate(image)
    op, sp, ts = ThinningSpeed(og_image, image*255, exec_time)
    print ("Name : ", imageName)
    print ("Algo : GH")
    print ("Thinning Rate : ", tr)
    print ("Object Points : ", op)
    print ("Skeleton Points : ", sp)
    print ("Thinning Speed : ", ts)

In [58]:
def modifiedK3M(image,colouredImage,imageName):
    def c1(i, j, image):
        wt = CalculateWeight(i, j, image)
        if (wt==95 and image[i-2][j]==0) or (wt==125 and image[i][j-2]==0) or (wt==215 and image[i][j+2]==0) or (wt==245 and image[i+2][j]==0):
            return True
        return False

    def c2(i, j, image, borders):
        wt = CalculateWeight(i, j, image)
        if wt==241 and image[i][j+1] in borders and image[i][j+2]==0 and image[i+1][j+2]==0:
            return True
        return False

    def c3(i, j, image, borders):
        wt = CalculateWeight(i, j, image)
        if (wt==195 or wt==227) and image[i+1][j-1] in borders:
            return True
        return False


    # name = "circle.png"
    # InvertImage(name(), cv2.imread(name(), 0))
    # exit()

    # INITIALISATION
    border_lookup = {3, 6, 7, 12, 14, 15, 24, 28, 30, 31, 48, 56, 60, 62,63, 96, 112, 120, 124, 126, 127, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223, 224, 225,227, 231, 239, 240, 241, 243, 247, 248, 249, 251,252, 253, 254,}
    phase1_lookup = {7, 14, 28, 56, 112, 131, 193, 224}
    phase2_lookup = {7, 14, 15, 24, 28, 30, 48, 56, 60, 112, 120, 131,135, 192, 193, 195, 224, 225, 240}
    phase3_lookup = {7, 14, 15, 28, 30, 31, 56, 60, 62, 112, 120, 124, 131, 135, 143, 193, 195, 199, 224, 225, 227, 240, 241, 248}
    phase4_lookup = {7, 14, 15, 28, 30, 31, 56, 60, 62, 63, 112, 120, 124, 126, 131, 135, 143, 159, 193, 195, 199, 207, 224, 225, 227, 231, 240, 241, 243, 248, 249, 252}
    phase5_lookup = {3, 6, 7, 12, 14, 15, 24, 28, 30, 31, 48, 56, 60, 62, 63, 96, 112, 120, 124, 126, 127, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223,224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254}
    one_pixel_lookup = {2, 5, 13, 20, 21, 22, 32, 48, 52, 54, 65, 67,69, 80, 81, 84, 88, 97, 99, 128, 133, 141, 208, 216}
    lookup_arrays = [phase1_lookup, phase2_lookup, phase3_lookup, phase4_lookup, phase5_lookup]
    change = True

    # BACKGROUND = 0, FOREGROUND = 1
#     image = cv2.imread(name(), 0)
    n, m = len(image), len(image[0])
    image[image<=128] = 0
    image[image>128] = 255
    image[image==255] = 1

    start = time.time()
    iteration = 1
    while change and iteration<100:
        change = False

        # IDENTIFYING BORDER PIXELS (Phase 0)
        borders = set()
        for i in range(n):
            for j in range(m):
                wt = CalculateWeight(i, j, image)
                if image[i][j] and CalculateWeight(i, j, image) in border_lookup:
                    if wt==193 and image[i-1][j-1]==1:
                        borders.add((i-1, j-1))
                    borders.add((i, j))
                else:
                    if c1(i, j, image):
                        borders.add((i, j))

        # Phase 0A
        for i in range(n):
            for j in range(m):
                if (i, j) in borders:
                    wt = CalculateWeight(i, j, image)
                    if wt==31 or wt==124:
                        borders.add((i, j))


        # REMOVING PIXELS IN PHASES
        ind = 0
        while ind<len(lookup_arrays):
            curr_lookup = lookup_arrays[ind]
            to_be_removed = set()
            for pixel in borders:
                wt = CalculateWeight(pixel[0], pixel[1], image)
                if wt in curr_lookup:
                    if c2(i, j, image, borders):
                        if CalculateWeight(pixel[0], pixel[1]+1, image) in curr_lookup:
                            image[pixel[0]][pixel[1]+1] = 0
                            to_be_removed.add((pixel[0], pixel[1]+1))
                    elif c3(i, j, image, borders):
                        if CalculateWeight(pixel[0]+1, pixel[1]-1, image) in curr_lookup:
                            image[pixel[0]+1][pixel[1]-1] = 0
                            to_be_removed.add((pixel[0]+1, pixel[1]-1))
                    change = True
                    image[pixel[0]][pixel[1]] = 0
                    to_be_removed.add((pixel[0], pixel[1]))


            for pixel in to_be_removed:
                borders.remove(pixel)
            ind += 1

        # VISUALISATION
        # image = image*255
        # cv2.imshow("image", image)
        # cv2.waitKey(100)
        # image = image//255


    # ONE PIXEL THINNING
    for i in range(n):
        for j in range(m):
            if image[i][j] and CalculateWeight(i, j, image) in one_pixel_lookup:
                image[i][j] = 0

    exec_time = time.time()-start
    print ("Execution Time For the Image : ", exec_time)

#     og_image = cv2.imread(name(), 0)
#     og_image[og_image<=128] = 0
#     og_image[og_image>128] = 255
#     image = image*255
#     final_image = og_image-image
#     cv2.imshow("image", final_image)
#     cv2.imwrite("Output/"+name()[7:-4]+"_modk3m.png", final_image)
#     cv2.waitKey(1000)

    image2 = image.copy()
    image2*=255
    mask_inv = cv2.bitwise_not(image2)

    og_image = colouredImage
    img1_bg = cv2.bitwise_and(og_image,og_image,mask = mask_inv)
    # og_image[og_image<=128] = 0
    # og_image[og_image>128] = 255

    image2 = cv2.cvtColor(image2,cv2.COLOR_GRAY2RGB)
    final_image = cv2.add(img1_bg,image2)
    # final_image = og_image-image
    
    print(image[image==1].shape)
    cv2.imwrite("modk3m"+imageName+".jpg",final_image)
#     cv2.imshow("Output",final_image)
#     cv2.imwrite(path+name[:-4]+"_kmm.png", final_image)
#     cv2.waitKey()
    
    og_image[og_image<=128] = 0
    og_image[og_image>128] = 1
    og_image *= 255
    tr = ThinningRate(image)
    op, sp, ts = ThinningSpeed(og_image, image*255, exec_time)
    print ("Name : ", imageName)
    print ("Algo : Modified K3M")
    print ("Thinning Rate : ", tr)
    print ("Object Points : ", op)
    print ("Skeleton Points : ", sp)
    print ("Thinning Speed : ", ts)

In [38]:
camel = cv2.imread("./camel.jpg");
camelbnw = cv2.imread("./camel_bnw.png",0);
camel = cv2.resize(camel,(256,256),interpolation=cv2.INTER_CUBIC)
camelbnw = cv2.resize(camelbnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [4]:
print(camel.shape)
print(camelbnw.shape)
print(camelbnw[camelbnw==255])

(256, 256, 3)
(256, 256)
[255 255 255 ... 255 255 255]


In [64]:
kmm(camelbnw.copy(),camel.copy(),"Camel")

Execution Time For the Image :  41.90102815628052
Name :  Camel
Algo : KMM
Thinning Rate :  0.9970242214532872
Object Points :  130820
Skeleton Points :  654
Thinning Speed :  3106.510883563832


In [66]:
k3m(camelbnw.copy(),camel.copy(),"Camel")

Execution Time For the Image :  6.583402156829834
(905,)
Name :  Camel
Algo : K3M
Thinning Rate :  0.9930334486735871
Object Points :  130820
Skeleton Points :  905
Thinning Speed :  19733.717750361335


In [41]:
zhangSuen(camelbnw.copy(),camel.copy(),"Camel")

Execution Time For the Image :  73.6437759399414
(667,)
Name :  Camel
Algo : ZS
Thinning Rate :  0.9959246443675509
Object Points :  130820
Skeleton Points :  667
Thinning Speed :  1767.3319752933835


In [42]:
guoHall(camelbnw.copy(),camel.copy(),"Camel")

Execution Time For the Image :  76.40997004508972
(662,)
Name :  Camel
Algo : GH
Thinning Rate :  0.9963860053825452
Object Points :  130820
Skeleton Points :  662
Thinning Speed :  1703.4164510625174


In [43]:
modifiedK3M(camelbnw.copy(),camel.copy(),"Camel")

Execution Time For the Image :  80.79793667793274
(655,)
Name :  Camel
Algo : Modified K3M
Thinning Rate :  0.996724336793541
Object Points :  130820
Skeleton Points :  655
Thinning Speed :  1610.994108907118


In [8]:
dog = cv2.imread("./dog.jpg");
dogbnw = cv2.imread("./dog_bnw.png",0);
dog = cv2.resize(dog,(256,256),interpolation=cv2.INTER_CUBIC)
dogbnw = cv2.resize(dogbnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [23]:
k3m(dogbnw.copy(),dog.copy(),"Dog")
print()
print("-----------------------------------------")
print()
kmm(dogbnw.copy(),dog.copy(),"Dog")
print()
print("-----------------------------------------")
print()
zhangSuen(dogbnw.copy(),dog.copy(),"Dog")
print()
print("-----------------------------------------")
print()
guoHall(dogbnw.copy(),dog.copy(),"Dog")
print()
print("-----------------------------------------")
print()
modifiedK3M(dogbnw.copy(),dog.copy(),"Dog")

Execution Time For the Image :  3.4355621337890625
(914,)
Name :  Dog
Algo : K3M
Thinning Rate :  0.9929334871203384
Object Points :  120539
Skeleton Points :  914
Thinning Speed :  34819.62931872993

-----------------------------------------

Execution Time For the Image :  24.133479356765747
Name :  Dog
Algo : KMM
Thinning Rate :  0.9979238754325259
Object Points :  120539
Skeleton Points :  601
Thinning Speed :  4969.776559233501

-----------------------------------------

Execution Time For the Image :  72.29448413848877
(516,)
Name :  Dog
Algo : ZS
Thinning Rate :  0.9970549788542868
Object Points :  120539
Skeleton Points :  516
Thinning Speed :  1660.1958148021572

-----------------------------------------

Execution Time For the Image :  75.04067516326904
(630,)
Name :  Dog
Algo : GH
Thinning Rate :  0.9969550173010381
Object Points :  120539
Skeleton Points :  630
Thinning Speed :  1597.9200578767332

-----------------------------------------

Execution Time For the Image :  8

In [24]:
hockey = cv2.imread("./hockey.jpg");
hockeybnw = cv2.imread("./hockey_bnw.png",0);
hockey = cv2.resize(hockey,(256,256),interpolation=cv2.INTER_CUBIC)
hockeybnw = cv2.resize(hockeybnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [27]:
k3m(hockeybnw.copy(),hockey.copy(),"Hockey Player")
print()
print("-----------------------------------------")
print()
kmm(hockeybnw.copy(),hockey.copy(),"Hockey Player")
print()
print("-----------------------------------------")
print()
zhangSuen(hockeybnw.copy(),hockey.copy(),"Hockey Player")
print()
print("-----------------------------------------")
print()
guoHall(hockeybnw.copy(),hockey.copy(),"Hockey Player")
print()
print("-----------------------------------------")
print()
modifiedK3M(hockeybnw.copy(),hockey.copy(),"Hockey Player")

Execution Time For the Image :  1.5241518020629883
(560,)
Name :  Hockey Player
Algo : K3M
Thinning Rate :  0.9957093425605537
Object Points :  90739
Skeleton Points :  560
Thinning Speed :  59166.67872448127

-----------------------------------------

Execution Time For the Image :  11.516629219055176
Name :  Hockey Player
Algo : KMM
Thinning Rate :  0.9977931564782776
Object Points :  90739
Skeleton Points :  442
Thinning Speed :  7840.575422068504

-----------------------------------------

Execution Time For the Image :  27.415040493011475
(456,)
Name :  Hockey Player
Algo : ZS
Thinning Rate :  0.9969627066512879
Object Points :  90739
Skeleton Points :  456
Thinning Speed :  3293.1922906703185

-----------------------------------------

Execution Time For the Image :  23.6483633518219
(440,)
Name :  Hockey Player
Algo : GH
Thinning Rate :  0.9977085736255287
Object Points :  90739
Skeleton Points :  440
Thinning Speed :  3818.403779432933

-----------------------------------------

In [30]:
multiple = cv2.imread("./multiple.jpg");
multiplebnw = cv2.imread("./multiple.jpg",0);
multiple = cv2.resize(multiple,(256,256),interpolation=cv2.INTER_CUBIC)
multiplebnw = cv2.resize(multiplebnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [31]:
k3m(multiplebnw.copy(),multiple.copy(),"Multiple")

Execution Time For the Image :  18.805585384368896
(4770,)
Name :  Multiple
Algo : K3M
Thinning Rate :  0.9629757785467128
Object Points :  168404
Skeleton Points :  4770
Thinning Speed :  8701.35104307956


In [33]:
motorbike = cv2.imread("./motorbike.jpg");
motorbikebnw = cv2.imread("./motorbike_bnw.png",0);
motorbike = cv2.resize(motorbike,(256,256),interpolation=cv2.INTER_CUBIC)
motorbikebnw = cv2.resize(motorbikebnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [35]:
k3m(motorbikebnw.copy(),motorbike.copy(),"motorbike")
print()
print("-----------------------------------------")
print()
kmm(motorbikebnw.copy(),motorbike.copy(),"motorbike")
print()
print("-----------------------------------------")
print()
zhangSuen(motorbikebnw.copy(),motorbike.copy(),"motorbike")
print()
print("-----------------------------------------")
print()
guoHall(motorbikebnw.copy(),motorbike.copy(),"motorbike")
print()
print("-----------------------------------------")
print()
modifiedK3M(motorbikebnw.copy(),motorbike.copy(),"motorbike")

Execution Time For the Image :  0.7680349349975586
(776,)
Name :  motorbike
Algo : K3M
Thinning Rate :  0.9939484813533257
Object Points :  73706
Skeleton Points :  776
Thinning Speed :  94956.61808695177

-----------------------------------------

Execution Time For the Image :  8.207962989807129
Name :  motorbike
Algo : KMM
Thinning Rate :  0.9974240676662822
Object Points :  73706
Skeleton Points :  568
Thinning Speed :  8910.61522704534

-----------------------------------------

Execution Time For the Image :  27.012981414794922
(636,)
Name :  motorbike
Algo : ZS
Thinning Rate :  0.9958631295655517
Object Points :  73706
Skeleton Points :  636
Thinning Speed :  2704.9957528930813

-----------------------------------------

Execution Time For the Image :  28.48382306098938
(567,)
Name :  motorbike
Algo : GH
Thinning Rate :  0.9972856593617839
Object Points :  73706
Skeleton Points :  567
Thinning Speed :  2567.7381804891584

-----------------------------------------

Execution Time

In [36]:
duck = cv2.imread("./duck.jpg");
duckbnw = cv2.imread("./duck_bnw.png",0);
duck = cv2.resize(duck,(256,256),interpolation=cv2.INTER_CUBIC)
duckbnw = cv2.resize(duckbnw,(256,256),interpolation=cv2.INTER_CUBIC)

In [52]:
k3m(duckbnw.copy(),duck.copy(),"duck")
print()
print("-----------------------------------------")
print()
kmm(duckbnw.copy(),duck.copy(),"duck")
print()
print("-----------------------------------------")
print()
zhangSuen(duckbnw.copy(),duck.copy(),"duck")
print()
print("-----------------------------------------")
print()
guoHall(duckbnw.copy(),duck.copy(),"duck")
print()
print("-----------------------------------------")
print()
modifiedK3M(duckbnw.copy(),duck.copy(),"duck")

Execution Time For the Image :  1.7481718063354492
(487,)
Name :  duck
Algo : K3M
Thinning Rate :  0.9962629757785467
Object Points :  41025
Skeleton Points :  487
Thinning Speed :  23188.79634889921

-----------------------------------------

Execution Time For the Image :  19.239938735961914
Name :  duck
Algo : KMM
Thinning Rate :  0.9981776239907728
Object Points :  41025
Skeleton Points :  368
Thinning Speed :  2113.156416865655

-----------------------------------------

Execution Time For the Image :  57.99445462226868
(387,)
Name :  duck
Algo : ZS
Thinning Rate :  0.9975932333717801
Object Points :  41025
Skeleton Points :  387
Thinning Speed :  700.7221684329081

-----------------------------------------

Execution Time For the Image :  54.70278024673462
(387,)
Name :  duck
Algo : GH
Thinning Rate :  0.9978700499807767
Object Points :  41025
Skeleton Points :  387
Thinning Speed :  742.8872868381459

-----------------------------------------

Execution Time For the Image :  61.

In [60]:
deletion_array = {3, 5, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31, 48, 52, 53, 54, 55, 56, 60, 61, 62, 63, 65, 67, 69, 71, 77, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 97, 99, 101, 103, 109, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 131, 133, 135, 141, 143, 149, 151, 157, 159, 181, 183, 189, 191, 192, 193, 195, 197, 199, 205, 207, 208, 209, 211, 212, 213, 214, 215, 216, 217, 219, 220, 221, 222, 223, 224, 225, 227, 229, 231, 237, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255}
border_lookup = {3, 6, 7, 12, 14, 15, 24, 28, 30, 31, 48, 56, 60, 62, 63, 96, 112, 120, 124, 126, 127, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223, 224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254}
phase1_lookup = {7, 14, 28, 56, 112, 131, 193, 224}
phase2_lookup = {7, 14, 15, 28, 30, 56, 60, 112, 120, 131, 135, 193, 195, 224, 225, 240}
phase3_lookup = {7, 14, 15, 28, 30, 31, 56, 60, 62, 112, 120, 124, 131, 135, 143, 193, 195, 199, 224, 225, 227, 240, 241, 248}
phase4_lookup = {7, 14, 15, 28, 30, 31, 56, 60, 62, 63, 112, 120, 124, 126, 131, 135, 143, 159, 193, 195, 199, 207, 224, 225, 227, 231, 240, 241, 243, 248, 249, 252}
phase5_lookup = {3, 6, 7, 12, 14, 15, 24, 28, 30, 31, 48, 56, 60, 62, 63, 96, 112, 120, 124, 126, 127, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223,224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254}
one_pixel_lookup = {3, 6, 7, 12, 14, 15, 24, 28, 30, 31, 48, 56, 60, 62, 63, 96, 112, 120, 124, 126, 127, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223,224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254}
lookup_arrays = [phase1_lookup, phase2_lookup, phase3_lookup, phase4_lookup, phase5_lookup]